In [1]:
from scipy.optimize         import newton
from sklearn.neighbors      import KDTree
from scipy.sparse           import csr_matrix
from scipy.sparse.csgraph   import dijkstra
from scipy.linalg           import eig
from sklearn.decomposition  import PCA
import numpy                as np
import gudhi                as gd
import warnings
import gurobipy             as gp
from gurobipy               import GRB
import matplotlib.pyplot    as plt
from kneed                  import KneeLocator
from scipy.spatial          import distance_matrix
from ripser                 import Rips
from sklearn.preprocessing  import PolynomialFeatures
import sympy                as sp
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.problem     import Problem
from pymoo.optimize         import minimize
import tadasets
import rml

In [9]:
m = gp.Model()
m.setParam('OutputFlag', 0)
m.setParam(GRB.Param.NonConvex, 2)
x = m.addMVar(shape=2, lb=float('-inf'))
z = m.addMVar(shape=2, lb=float('-inf'))

Q = A.T @ A
c = -2 * y.T @ A
obj = x @ Q @ x + c @ x + y.T @ y

m.setObjective(obj, GRB.MINIMIZE)

c0 = m.addConstr(x@x == alpha**2, name="c0")

c1 = m.addConstr(z == x+b_prime-c_1_prime, name="d0")
d1 = m.addConstr(z@z == (q-c_1)@(q-c_1), name="d0")

m.feasRelax(relaxobjtype=1, minrelax=False, vars=None, lbpen=None, ubpen=None, constrs=[d1], rhspen=[1])
m.optimize()

In [2]:
params = {'max_components':5, 'S':0.5, 'k':10, 'threshold_var':0.05, 'edge_sen':1, 'k0':2, 'beta':100}  # change edge sen
sphere5 = tadasets.dsphere(n=int(1500*1.3))
sphere5 = sphere5[sphere5[:, 2]>=-0.7]
S = rml.Simplex()
S.build_simplex(sphere5, **params)

In [3]:
A, y, alpha, c_1, c_1_prime, q, b_prime = S.normal_coords_trade(**params)

Restricted license - for non-production use only - expires 2023-10-25
1656
0


ValueError: not enough values to unpack (expected 7, got 2)

In [13]:
print(A)
print(y)
print(alpha)
print(c_1)
print(c_1_prime)
print(q)
print(b_prime)

[[17.66549616  4.88347078]
 [16.3030199  -8.37434078]
 [17.66485584  4.88578647]
 [18.29176392 -1.15300225]]
[ 0.27639128 -0.88839732  0.39363632 -0.63422566]
0.05456112735778344
[-0.41712114 -0.34422138  0.84114303]
[0.11388834 0.4564227 ]
[-0.48384386 -0.23117099  0.84407055]
[-0.07370518  0.40456412]


In [20]:
m = gp.Model()
m.setParam('OutputFlag', 0)
m.setParam(GRB.Param.NonConvex, 2)
x = m.addMVar(shape=2, lb=float('-inf'))
z = m.addMVar(shape=2, lb=float('-inf'))

Q = A.T @ A
c = -2 * y.T @ A
obj = x @ Q @ x + c @ x + y.T @ y

m.setObjective(obj, GRB.MINIMIZE)

c0 = m.addConstr(x@x == alpha**2, name="c0")

c1 = m.addConstr(z == x+b_prime-c_1_prime, name="d0")
d1 = m.addConstr(z@z == (q-c_1)@(q-c_1), name="d0")

m.feasRelax(relaxobjtype=1, minrelax=False, vars=None, lbpen=None, ubpen=None, constrs=[c0, d1], rhspen=[10, 100])

m.optimize()

In [21]:
x.X

array([0.05240733, 0.01519557])

In [22]:
m.status

2

In [23]:
np.asarray(x.X) @ np.asarray(x.X)

0.002977433436283547

In [24]:
alpha**2

0.002976916618552265